## Crawl ảnh

In [1]:
import requests
import urllib.parse
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tabulate import tabulate  # pip install tabulate

# ---- 1. Đọc danh sách tên khoa học từ CSV ----
csv_path = "flower.csv"
df_names = pd.read_csv(csv_path, encoding="utf-8")
scientific_names = df_names["ten_khoa_hoc"].tolist()

print("Tổng số loài cần kiểm tra:", len(scientific_names))

# ---- 2. Hàm kiểm tra trên iNaturalist với retry ----
def check_inaturalist(scientific_name, max_retries=3):
    retries = 0
    last_error = ""

    while retries < max_retries:
        try:
            name_encoded = urllib.parse.quote(scientific_name)
            url = f"https://api.inaturalist.org/v1/taxa?q={name_encoded}&rank=species"

            response = requests.get(url, timeout=10)

            if response.status_code != 200:
                last_error = f"HTTP {response.status_code}"
                retries += 1
                time.sleep(1)
                continue

            try:
                data = response.json()
            except ValueError as e:
                last_error = f"JSONDecodeError: {e}"
                retries += 1
                time.sleep(1)
                continue

            results = data.get("results", [])
            if results:
                best_match = results[0]
                matched_name = best_match.get("name", "")
                return scientific_name, True, matched_name, ""
            else:
                return scientific_name, False, "", ""

        except Exception as e:
            last_error = str(e)
            retries += 1
            time.sleep(1)

    return scientific_name, f"Lỗi: {last_error}", "", last_error

# ---- 3. Chạy đa luồng ----
results = []
max_threads = 1
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    future_to_name = {executor.submit(check_inaturalist, name): name for name in scientific_names}

    for future in as_completed(future_to_name):
        name, exists, matched_name, error = future.result()
        results.append({
            "Tên khoa học": name,
            "Tồn tại trên iNaturalist": exists,
            "Tên khớp": matched_name,
            "Lỗi": error
        })

# ---- 4. Chuyển sang DataFrame và in bảng ----
df_result = pd.DataFrame(results)

# In ra console dưới dạng bảng đẹp
print("\n=== Kết quả kiểm tra iNaturalist ===")
print(tabulate(df_result, headers='keys', tablefmt='grid', showindex=False))

Tổng số loài cần kiểm tra: 113

=== Kết quả kiểm tra iNaturalist ===
+---------------------------+----------------------------+---------------------------+-------+
| Tên khoa học              | Tồn tại trên iNaturalist   | Tên khớp                  | Lỗi   |
+===========================+============================+===========================+=======+
| Aconitum napellus         | True                       | Aconitum napellus         |       |
+---------------------------+----------------------------+---------------------------+-------+
| Adenium obesum            | True                       | Adenium obesum            |       |
+---------------------------+----------------------------+---------------------------+-------+
| Alpinia purpurata         | True                       | Alpinia purpurata         |       |
+---------------------------+----------------------------+---------------------------+-------+
| Alstroemeria aurea        | True                       | Alstroemeria aure

In [ ]:
import os
import requests
import pandas as pd
from time import sleep
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

In [ ]:
csv_path = "1.csv"
root_folder = "Vietnam_flower"
os.makedirs(root_folder, exist_ok=True)
df = pd.read_csv(csv_path)
MAX_IMAGES = 300

In [ ]:
def download_image(photo_url, file_path):
    try:
        img_data = requests.get(photo_url, timeout=30).content
        with open(file_path, "wb") as f:
            f.write(img_data)
        return True
    except Exception as e:
        print(f"  Lỗi tải ảnh: {e}")
        return False

In [ ]:
for species in df["ten_khoa_hoc"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    page = 1
    total_images = len(os.listdir(species_folder))
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = (
            "https://api.inaturalist.org/v1/observations"
            f"?taxon_name={quote(species)}"
            "&per_page=200"
            f"&page={page}"
        )
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    for idx, p in enumerate(obs.get("photos", [])):
                        if total_images >= MAX_IMAGES:   # ✅ chặn ngay khi đạt 1000
                            stop = True
                            break

                        photo_url = p.get("url")
                        if not photo_url:
                            continue
                        photo_url = photo_url.replace("square", "original")

                        file_name = f"{obs['id']}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)

                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1   # ✅ tăng ngay khi submit task

                    if stop:
                        break

                # chờ tất cả job của vòng này xong
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Trang {page}: tổng {total_images} ảnh")
            page += 1
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")

In [ ]:
for species in df["ten_khoa_hoc"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    page = 1
    total_images = len(os.listdir(species_folder))
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = (
            "https://api.inaturalist.org/v1/observations"
            f"?taxon_name={quote(species)}"
            "&per_page=200"
            f"&page={page}"
        )
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    for idx, p in enumerate(obs.get("photos", [])):
                        if total_images >= MAX_IMAGES:   # ✅ chặn ngay khi đạt 1000
                            stop = True
                            break

                        photo_url = p.get("url")
                        if not photo_url:
                            continue
                        photo_url = photo_url.replace("square", "original")

                        file_name = f"{obs['id']}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)

                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1   # ✅ tăng ngay khi submit task

                    if stop:
                        break

                # chờ tất cả job của vòng này xong
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Trang {page}: tổng {total_images} ảnh")
            page += 1
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")